## Generative Local Metric Learning example

In [1]:
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Gaussian data generation

1) Make two true mean parameters

2) Make two random positive definite symmetric matrices for two true covariance matrices

In [2]:
dim = 10;

# True density
# Mean vectors
trueMean_1 = np.zeros(dim)
trueMean_2 = np.concatenate(([1],np.zeros(dim-1)))

# covariance matrices
trueCov_1 = np.random.randn(dim,dim)
trueCov_1 = np.dot(trueCov_1.T, trueCov_1)
trueCov_2 = np.random.randn(dim,dim)
trueCov_2 = trueCov_1 + .001*np.dot(trueCov_2.T, trueCov_2)   # two covariance matrices are very similar


### Generate data according to the true parameters

In [3]:
# data generation
datanum = 100
tstdatanum = 1000

X1 = np.random.multivariate_normal(trueMean_1, trueCov_1, datanum)
tstX1 = np.random.multivariate_normal(trueMean_1, trueCov_1, tstdatanum)

X2 = np.random.multivariate_normal(trueMean_2, trueCov_2, datanum)
tstX2 = np.random.multivariate_normal(trueMean_2, trueCov_2, tstdatanum)


### We are given data. Now estimate parameters

In [4]:
estM1 = sum(X1,axis=0)/datanum
estM2 = sum(X2,axis=0)/datanum

estCov1 = np.dot(X1.T,X1)/datanum - np.outer(estM1, estM1)
estCov2 = np.dot(X2.T,X2)/datanum - np.outer(estM2, estM2)


### Define a function for Gaussian probability densities

In [5]:
def getLogGaussian(X, mu, Sig):
    ''' evalute log probability of Gaussian distribution
    given the Gaussian distribution model (mean and covariance).'''
    # X: datanum x dim
    nData, nDim = X.shape
    detSig = np.linalg.det(Sig)
    invSig = np.linalg.inv(Sig)    
    logPs = np.zeros(nData)
    
    const = - nDim / (2.) * np.log(2*np.pi) - 1 / (2.) * np.log(detSig) # sacalar
    for i in range(nData):
        x = X[i,:]
        expval = - 1/2. * (x - mu).reshape(1,-1).dot(invSig).dot((x - mu).reshape(-1,1)) # 2d array
        logPs[i] =  const + expval
    return logPs


### Define a function for GLML metric

In [6]:
def GLML_Gaussian(tstpt, logPsAttstX, means, invSigs, regSup=-10):
    # Author: Yung-Kyun Noh
    # X -> L'*X, 
    # dist^2 = (x1 - x2)*L*L'*(x1 - x2)

    relPsAttstX = np.exp(logPsAttstX - np.max(logPsAttstX))  # 1,...,classnum
    
    classnum = np.shape(means)[0]
    Dim = np.shape(means)[1]

    # pre-calculation
    HessianOverPs = np.zeros([classnum,dim,dim])
    for iclass in range(classnum):
        invSigsDiff = np.dot(invSigs[iclass,:,:], tstpt - means[iclass,:])
        HessianOverPs[iclass,:,:] = np.outer(invSigsDiff,invSigsDiff) - invSigs[iclass,:,:]

    Met = np.zeros([Dim,Dim])
    ZeroVal = np.power(10,-100)
    ZeroPIndex = (abs(relPsAttstX) <= ZeroVal)    # 1 ~ classnum
    if np.sum(ZeroPIndex) == classnum:     # p1 = 0, p2 = 0, p3 = 0
        print '**** All p vals are zero ****'
        return
    elif np.sum(ZeroPIndex) == classnum - 1:    # p1 > 0, p2 = 0, p3 = 0
        for iclass in range(classnum):
            print ZeroPIndex[iclass]
            if not ZeroPIndex[iclass]:    # nonZero P
                Met = Met - (classnum - 1)*HessianOverPs[iclass,:,:]
            else:
                Met = Met + HessianOverPs[iclass,:,:]
    else:    # general case: more than two classes have positive probability density
        for iclass in range(classnum):
            # to avoid numerical error
            noniclass = \
                np.concatenate([np.arange(iclass),iclass+1 + np.arange(classnum - (iclass+1))])
            sumPsqExCur = np.sum(np.power(relPsAttstX[noniclass],2))
            sumPExCur = np.sum(relPsAttstX[noniclass])
            Met = Met + \
                (sumPsqExCur*relPsAttstX[iclass] - sumPExCur*np.power(relPsAttstX[iclass],2))* \
                HessianOverPs[iclass,:,:];

    Met = (Met.T + Met)/2    # make sure the matrix is symmetric

    vals, vecs = np.linalg.eig(Met)
    sortedIdxes = np.argsort(vals)
    evals = vals[sortedIdxes]
    evecs = vecs[:,sortedIdxes]

    PosEvalIndex = (evals >= 0)
    NegEvalIndex = (evals < 0)

    regR = Dim*np.power(10,regSup)*np.max(np.abs(evals[[0,-1]]))

    PosEvalNum = np.sum(PosEvalIndex)
    NegEvalNum = np.sum(NegEvalIndex)
   
    evalscale = -1/Dim*( \
        np.sum(log(evals[PosEvalIndex]*PosEvalNum + regR)) + \
        np.sum(log(evals[NegEvalIndex]*NegEvalNum*(-1) + regR)))

    evals = evals*exp(evalscale);
    regR = regR*exp(evalscale);

    L = np.hstack( \
        [evecs[:,PosEvalIndex].dot(np.diag(sqrt(evals[PosEvalIndex]*PosEvalNum + regR))), \
        evecs[:,NegEvalIndex].dot(np.diag(sqrt(evals[NegEvalIndex]*NegEvalNum*(-1) + regR)))])
    return L


### Classification

In [7]:
classnum = 2
knum = 5
means = np.vstack([estM1,estM2])
invSigs = np.zeros([classnum,dim,dim])
invSigs[0,:,:] = np.linalg.inv(estCov1)
invSigs[1,:,:] = np.linalg.inv(estCov2)

probC1tst1 = getLogGaussian(tstX1, estM1, estCov1)   # array
probC2tst1 = getLogGaussian(tstX1, estM2, estCov2)
probC1tst2 = getLogGaussian(tstX2, estM1, estCov1)
probC2tst2 = getLogGaussian(tstX2, estM2, estCov2)

totX = np.vstack([X1,X2])
Labels = np.concatenate((np.ones(datanum), 2*np.ones(datanum)))

# for class 1
AlllogPsAttstX1 = np.transpose([probC1tst1,probC2tst1])

Idxes = np.zeros([tstdatanum,knum], dtype=np.int)
for icnt in range(tstdatanum):
    L = GLML_Gaussian(tstX1[icnt,:], AlllogPsAttstX1[icnt,:], means, invSigs)

    dists = np.diag(np.dot(np.dot(totX, L), np.dot(totX, L).T)) + \
        np.dot(np.dot(tstX1[icnt,:], L), np.dot(tstX1[icnt,:], L))*np.ones(datanum*2) - \
        2*np.dot(np.dot(totX, L), np.dot(tstX1[icnt,:], L))
    Idxes[icnt,:] = np.argsort(dists)[0:knum]

NNEachClass = np.zeros([np.shape(tstX1)[0], classnum])
for iclass in range(classnum):
    NNEachClass[:,iclass] = np.sum(Labels[Idxes[:,0:knum]] == (iclass + 1), axis = 1)

obtainedClasses = np.argmax(NNEachClass, axis = 1)
class1CorrectNum = np.double(sum(obtainedClasses + 1 == 1))   # class 1


# class 2
AlllogPsAttstX2 = np.transpose([probC1tst2,probC2tst2])

Idxes = np.zeros([tstdatanum,knum], dtype=np.int)
for icnt in range(tstdatanum):
    L = GLML_Gaussian(tstX2[icnt,:], AlllogPsAttstX2[icnt,:], means, invSigs)

    dists = np.diag(np.dot(np.dot(totX, L), np.dot(totX, L).T)) + \
        np.dot(np.dot(tstX2[icnt,:], L), np.dot(tstX2[icnt,:], L))*np.ones(datanum*2) - \
        2*np.dot(np.dot(totX, L), np.dot(tstX2[icnt,:], L))
    Idxes[icnt,:] = np.argsort(dists)[0:knum]

NNEachClass = np.zeros([np.shape(tstX2)[0], classnum])
for iclass in range(classnum):
    NNEachClass[:,iclass] = np.sum(Labels[Idxes[:,0:knum]] == (iclass + 1), axis = 1)

obtainedClasses = np.argmax(NNEachClass, axis = 1)
class2CorrectNum = np.double(sum(obtainedClasses + 1 == 2))   # class 2

GLMLAccuracy = (class1CorrectNum + class2CorrectNum)/(2.*tstdatanum)

print class1CorrectNum
print class2CorrectNum
print GLMLAccuracy

908.0
950.0
0.929
